# Data Cleaning - AQI 

In [2_data_aquisition_aqi.ipynb](2_data_aquisition_aqi.ipynb), we've collected AQI data from all the monitoring stations in the San Joaquin county of the state of California. The goal of this notebook is to condense these daily summaries of AQI data from multiple stations in the county to an yearly estimate of AQI in the city Stockton, CA.

This would facilitate us to easily analyse how the estimated AQI index of Stockton city has changed over the past 60 years. Creating this yearly aggregate would also allow us to draw comparisons with an yearly aggregate of smoke estimates that we'll calculate in [4_smoke_estimates.ipynb](4_smoke_estimates.ipynb)

### Preliminaries
First we start with some imports statements

In [1]:
import pandas as pd
import polars as pl
from pyproj import Geod

We proceed to load the data that was generated in the previous step, i.e, [2_data_aquisition_aqi.ipynb](2_data_aquisition_aqi.ipynb). This data would contain the daily summaries of the gaseous and particle data from the monitoring stations in the county of interest, for the last 60 years.

In [2]:
# Loading the gaseous and particulate data
gaseous_data = pl.from_dataframe( pd.read_csv("generated_files/intermediate/gaseous_AQI_1964-2024.csv") )

particulate_data = pl.from_dataframe( pd.read_csv("generated_files/intermediate/particulate_AQI_1964-2024.csv") )

# merging both gaseous and particle data, since they have the same schema
aqi_data = pl.concat([gaseous_data, particulate_data])
print(aqi_data)
print(aqi_data.columns)

shape: (218_695, 32)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ state_cod ┆ county_co ┆ site_numb ┆ parameter ┆ … ┆ city     ┆ cbsa_code ┆ cbsa      ┆ date_of_l │
│ e         ┆ de        ┆ er        ┆ _code     ┆   ┆ ---      ┆ ---       ┆ ---       ┆ ast_chang │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str      ┆ i64       ┆ str       ┆ e         │
│ i64       ┆ i64       ┆ i64       ┆ i64       ┆   ┆          ┆           ┆           ┆ ---       │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆           ┆ str       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 6         ┆ 77        ┆ 1002      ┆ 42101     ┆ … ┆ Stockton ┆ 44700     ┆ Stockton- ┆ 2016-03-3 │
│           ┆           ┆           ┆           ┆   ┆          ┆           ┆ Lodi, CA  ┆ 1         │
│ 6         ┆ 77        ┆ 1002      ┆ 42101     ┆ … ┆ Stockton ┆ 44700

### Exploring the montitoring stations

As mentioned previously, we have the AQI data coming from multiple Air quality monitoring stations. Let's find the unique stations from where our data is coming

In [3]:
print("Unique stations - -")

unique_stations = (
    aqi_data
    .select(
        [
            'state_code',
            'county_code',
            'site_number',
            'latitude',
            'longitude'
        ]
    )
    .unique()
)
print(unique_stations)

Unique stations - -
shape: (13, 5)
┌────────────┬─────────────┬─────────────┬───────────┬─────────────┐
│ state_code ┆ county_code ┆ site_number ┆ latitude  ┆ longitude   │
│ ---        ┆ ---         ┆ ---         ┆ ---       ┆ ---         │
│ i64        ┆ i64         ┆ i64         ┆ f64       ┆ f64         │
╞════════════╪═════════════╪═════════════╪═══════════╪═════════════╡
│ 6          ┆ 77          ┆ 2           ┆ 38.116586 ┆ -121.286337 │
│ 6          ┆ 77          ┆ 8           ┆ 37.995201 ┆ -121.309115 │
│ 6          ┆ 77          ┆ 9           ┆ 37.906869 ┆ -121.148275 │
│ 6          ┆ 77          ┆ 3010        ┆ 38.029626 ┆ -121.354026 │
│ 6          ┆ 77          ┆ 5           ┆ 37.84826  ┆ -121.446892 │
│ …          ┆ …           ┆ …           ┆ …         ┆ …           │
│ 6          ┆ 77          ┆ 3003        ┆ 37.737985 ┆ -121.535503 │
│ 6          ┆ 77          ┆ 1003        ┆ 37.961578 ┆ -121.281414 │
│ 6          ┆ 77          ┆ 3002        ┆ 37.739929 ┆ -121.533002 │

We see that we have data from 13 AQI stations, each of them situated at different locations in and around the county. Since we want to estimate the AQI in Stockton using data from all these stations, we need to find a way to agrregate the data. 

One way to proceed is to take an average of all the AQIs recorded, to find the AQI at the city; but this would mean that an index recorded at a monitoring station 10 miles from the city will be given the same weight as that from a station in the city. This might not be right, since the second monitoring station will be providing more accurate data for the city.

To help with this, we can use a weighted average, where the AQI from the closest station is given more preference. So lets first start with calculating distances from Stockton, CA to all the monitoring stations.

In [4]:
# calculate distances for each station from the city

geodcalc = Geod(ellps='WGS84')
CITY_LOCATION = {
    'city'   : 'Stockton',
    'latlon' : [37.975556, -121.300833] 
}


def distance_in_miles(row):
    # use a predefined function to calculate the arc distance between 2 geo points.
    d = geodcalc.inv(CITY_LOCATION["latlon"][1],CITY_LOCATION["latlon"][0],row["longitude"],row["latitude"])
    # convert the resulting distance to miles
    distance_in_miles = d[2]*0.00062137
    return distance_in_miles

# run each record of the unique_stations list though the above function, to calculate the distances
# add this calculated distance as a column to the dataframe
unique_stations_dist = (
    unique_stations
    .with_columns(
        pl.struct(["latitude", "longitude"])
        .map_elements(
            distance_in_miles,
            return_dtype=pl.Float64
        ).alias("distance_in_miles")
    )
    .sort("distance_in_miles")
)

print(unique_stations_dist)

shape: (13, 6)
┌────────────┬─────────────┬─────────────┬───────────┬─────────────┬───────────────────┐
│ state_code ┆ county_code ┆ site_number ┆ latitude  ┆ longitude   ┆ distance_in_miles │
│ ---        ┆ ---         ┆ ---         ┆ ---       ┆ ---         ┆ ---               │
│ i64        ┆ i64         ┆ i64         ┆ f64       ┆ f64         ┆ f64               │
╞════════════╪═════════════╪═════════════╪═══════════╪═════════════╪═══════════════════╡
│ 6          ┆ 77          ┆ 8           ┆ 37.995201 ┆ -121.309115 ┆ 1.428345          │
│ 6          ┆ 77          ┆ 1003        ┆ 37.961578 ┆ -121.281414 ┆ 1.433032          │
│ 6          ┆ 77          ┆ 4           ┆ 37.954075 ┆ -121.289634 ┆ 1.602768          │
│ 6          ┆ 77          ┆ 1002        ┆ 37.950741 ┆ -121.268523 ┆ 2.457991          │
│ 6          ┆ 77          ┆ 3010        ┆ 38.029626 ┆ -121.354026 ┆ 4.725914          │
│ …          ┆ …           ┆ …           ┆ …         ┆ …           ┆ …                 │
│ 6   

Now, lets quickly analyse the data we have in the aqi_data dataframe. This is to understand what are the columns in the dataset that we can use for our AQI estimate in the city.

In [5]:
aqi_data.columns

['state_code',
 'county_code',
 'site_number',
 'parameter_code',
 'poc',
 'latitude',
 'longitude',
 'datum',
 'parameter',
 'sample_duration_code',
 'sample_duration',
 'pollutant_standard',
 'date_local',
 'units_of_measure',
 'event_type',
 'observation_count',
 'observation_percent',
 'validity_indicator',
 'arithmetic_mean',
 'first_max_value',
 'first_max_hour',
 'aqi',
 'method_code',
 'method',
 'local_site_name',
 'site_address',
 'state',
 'county',
 'city',
 'cbsa_code',
 'cbsa',
 'date_of_last_change']

In [6]:
# validity_indicator field seems to represent the validity of an entry
# Looking at the possible values for this field and the corresponding counts
aqi_data.group_by("validity_indicator").len()

validity_indicator,len
str,u32
"""N""",4228
"""Y""",214467


In [10]:
# looking at how sample durations are for each parameter (gaseous or particulate) and checking how it changes across monitoring sites
aqi_data.group_by([
            'site_number',
            'sample_duration',
            'parameter'
        ]).len()

site_number,sample_duration,parameter,len
i64,str,str,u32
2,"""8-HR RUN AVG BEGIN HOUR""","""Ozone""",1850
1002,"""3-HR BLK AVG""","""Sulfur dioxide""",1440
3002,"""1 HOUR""","""Ozone""",119
1003,"""1 HOUR""","""Ozone""",344
3002,"""1 HOUR""","""Nitrogen dioxide (NO2)""",218
…,…,…,…
3005,"""8-HR RUN AVG BEGIN HOUR""","""Ozone""",9998
3005,"""1 HOUR""","""Ozone""",3330
1002,"""8-HR RUN AVG BEGIN HOUR""","""Ozone""",24035


Looking at the daily AQI summary data, we notice that each station records data for multiple parameters on each day. For each parameter, we see AQI recordings for different sample durations. Since we are interested in aggregating this granular data to a year level, lets start by dropping all the invalid rows and finding the mean AQI per station per day per parameter.

In [11]:
# find the mean aqi per site per gas per day
aqi_mean_granular = (
    aqi_data
    .filter([
        # filter only for valid data
        pl.col("validity_indicator") == "Y", # ~4k records have validity indicator as "N"
        pl.col("aqi").is_not_null() # ~50k records do not have AQI info (.i.e 20%)
    ])
    .select(
        # select relevant features
        [
            'site_number',
            'parameter',
            'date_local',
            'aqi'
        ]
    )
    .group_by( 
        [
            'site_number',
            'date_local',
            'parameter'
        ]
    )
    .agg(pl.mean("aqi"))
)
print(aqi_mean_granular)

shape: (62_158, 4)
┌─────────────┬────────────┬──────────────────────────┬──────┐
│ site_number ┆ date_local ┆ parameter                ┆ aqi  │
│ ---         ┆ ---        ┆ ---                      ┆ ---  │
│ i64         ┆ str        ┆ str                      ┆ f64  │
╞═════════════╪════════════╪══════════════════════════╪══════╡
│ 1002        ┆ 1993-07-23 ┆ Carbon monoxide          ┆ 11.0 │
│ 1002        ┆ 1996-06-18 ┆ Nitrogen dioxide (NO2)   ┆ 36.0 │
│ 1002        ┆ 1974-07-08 ┆ Carbon monoxide          ┆ 24.0 │
│ 9           ┆ 1997-08-20 ┆ Ozone                    ┆ 31.0 │
│ 1002        ┆ 2006-10-08 ┆ Carbon monoxide          ┆ 13.0 │
│ …           ┆ …          ┆ …                        ┆ …    │
│ 1002        ┆ 1996-06-23 ┆ Carbon monoxide          ┆ 9.0  │
│ 1002        ┆ 2013-10-29 ┆ Ozone                    ┆ 26.0 │
│ 1002        ┆ 2019-07-22 ┆ Carbon monoxide          ┆ 2.0  │
│ 1002        ┆ 2011-07-25 ┆ PM2.5 - Local Conditions ┆ 36.0 │
│ 2           ┆ 1981-08-13 ┆ Ozone  

Now that we have an AQI value per station, per gas, per day, lets aggregate this further to get an AQI for the air, i.e, combine the individual gaseous AQI. For this, I decided to go with assigning the air AQI to the maximum of the pollutants AQIs.

This aggregation will give us an AQI value per station per day.

In [12]:
# calculate the max aqi recorded at a station on a given day

# daily AQI is given by the formula as max of all pollutants' AQI 

aqi_max_per_station = (
    aqi_mean_granular
    .group_by(
        [
            'site_number',
            'date_local'
        ]
    )
    .agg(
        pl.max('aqi'),
    )
)
aqi_max_per_station

site_number,date_local,aqi
i64,str,f64
3005,"""2006-10-27""",60.0
3010,"""2000-07-29""",20.0
1002,"""1995-06-06""",28.0
9,"""1986-05-13""",44.0
1002,"""2007-10-02""",48.0
…,…,…
2010,"""2018-10-31""",43.0
3003,"""1995-07-31""",129.0
1002,"""1983-05-23""",44.0


Now that we have the AQI per station per day, lets aggregate this even further to find an AQI value per station per year. For this, we just take the average of the AQI values of all the days in the year, by utilizing the inbuilt mean() function.

In [13]:
# find yearly AQI from each station by taking average

aqi_yearly_per_station = (
    aqi_max_per_station
    .with_columns(
        pl.col("date_local").str.strptime(pl.Date, "%Y-%m-%d").dt.year().alias("year")
    )
    .group_by(
        [
            "site_number",
            "year"
        ]
    )
    .agg(
        pl.mean("aqi")
    )
)

print(aqi_yearly_per_station)

shape: (164, 3)
┌─────────────┬──────┬───────────┐
│ site_number ┆ year ┆ aqi       │
│ ---         ┆ ---  ┆ ---       │
│ i64         ┆ i32  ┆ f64       │
╞═════════════╪══════╪═══════════╡
│ 9           ┆ 1990 ┆ 62.087798 │
│ 3005        ┆ 2013 ┆ 52.153846 │
│ 1002        ┆ 2005 ┆ 49.597826 │
│ 1002        ┆ 1981 ┆ 58.211749 │
│ 1002        ┆ 2011 ┆ 50.038043 │
│ …           ┆ …    ┆ …         │
│ 9           ┆ 1997 ┆ 48.462247 │
│ 3010        ┆ 2009 ┆ 21.064516 │
│ 3010        ┆ 2007 ┆ 21.333333 │
│ 3010        ┆ 2010 ┆ 15.933333 │
│ 9           ┆ 1993 ┆ 60.516575 │
└─────────────┴──────┴───────────┘


Now we have a dataframe containing the aggregated AQI values per station per year. As discussed previously, we want to take the weighted average of the AQIs from different stations based on their distance from the city.

To allow for this weighted average calculation, we join the `aqi_yearly_per_station` dataframe with table containing distances from the city to the monitoring station `unique_stations_dist` that was calculated above.

In [14]:
# join the yearly aggregated per station table with the unique_station table to map the distances
yearly_aqi_weighted_avg = (
    aqi_yearly_per_station
    .join(
        unique_stations_dist,
        on = "site_number"
    )
    .drop(
        [
            "state_code",
            "county_code",
            "latitude",
            "longitude"
        ]
    )
    .with_columns(
        (1 / pl.col("distance_in_miles")).alias("weight")  # Invert distance to create weight
    ) 
    .with_columns(
        (pl.col("aqi") * pl.col("weight")).alias("weighted_aqi")  # AQI * weight
    ) 
    .group_by("year")
    .agg(
        [
          (pl.sum("weighted_aqi") / pl.sum("weight")).round(2).alias("weighted_avg_aqi")  # Weighted average AQI
        ]
    )
)
yearly_aqi_weighted_avg = yearly_aqi_weighted_avg.sort(by="year")
print(yearly_aqi_weighted_avg)

shape: (61, 2)
┌──────┬──────────────────┐
│ year ┆ weighted_avg_aqi │
│ ---  ┆ ---              │
│ i32  ┆ f64              │
╞══════╪══════════════════╡
│ 1964 ┆ 32.41            │
│ 1965 ┆ 25.73            │
│ 1966 ┆ 21.06            │
│ 1967 ┆ 26.93            │
│ 1968 ┆ 25.57            │
│ …    ┆ …                │
│ 2020 ┆ 59.25            │
│ 2021 ┆ 52.98            │
│ 2022 ┆ 43.03            │
│ 2023 ┆ 45.11            │
│ 2024 ┆ 38.37            │
└──────┴──────────────────┘


In [15]:
yearly_aqi_weighted_avg.write_csv("generated_files/intermediate/yearly_weighted_aqi_1964-2024.csv")